In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.types import ArrayType, StringType

spark = SparkSession.builder.appName('my app').master('local').getOrCreate()
sc = spark.sparkContext

22/11/03 17:04:36 WARN Utils: Your hostname, bagdoyeong-ui-MacBookAir.local resolves to a loopback address: 127.0.0.1; using 192.168.1.30 instead (on interface en0)
22/11/03 17:04:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/03 17:04:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/11/03 17:04:39 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [45]:
from pyspark.ml.feature import PCA
from pyspark.ml.clustering import KMeans, GaussianMixture
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml.feature import VectorAssembler

import matplotlib.pyplot as plt
import numpy as np
import random

In [9]:
data.columns

['EXT1', 'country']

In [10]:
data = spark.read.format('csv').option('delimiter', '\t').option('header', 'true').load('./input_week10/IPIP-FFM-data-8Nov2018/data-final.csv')
data_col = data.columns

In [13]:
data_col

['EXT1',
 'EXT2',
 'EXT3',
 'EXT4',
 'EXT5',
 'EXT6',
 'EXT7',
 'EXT8',
 'EXT9',
 'EXT10',
 'EST1',
 'EST2',
 'EST3',
 'EST4',
 'EST5',
 'EST6',
 'EST7',
 'EST8',
 'EST9',
 'EST10',
 'AGR1',
 'AGR2',
 'AGR3',
 'AGR4',
 'AGR5',
 'AGR6',
 'AGR7',
 'AGR8',
 'AGR9',
 'AGR10',
 'CSN1',
 'CSN2',
 'CSN3',
 'CSN4',
 'CSN5',
 'CSN6',
 'CSN7',
 'CSN8',
 'CSN9',
 'CSN10',
 'OPN1',
 'OPN2',
 'OPN3',
 'OPN4',
 'OPN5',
 'OPN6',
 'OPN7',
 'OPN8',
 'OPN9',
 'OPN10',
 'EXT1_E',
 'EXT2_E',
 'EXT3_E',
 'EXT4_E',
 'EXT5_E',
 'EXT6_E',
 'EXT7_E',
 'EXT8_E',
 'EXT9_E',
 'EXT10_E',
 'EST1_E',
 'EST2_E',
 'EST3_E',
 'EST4_E',
 'EST5_E',
 'EST6_E',
 'EST7_E',
 'EST8_E',
 'EST9_E',
 'EST10_E',
 'AGR1_E',
 'AGR2_E',
 'AGR3_E',
 'AGR4_E',
 'AGR5_E',
 'AGR6_E',
 'AGR7_E',
 'AGR8_E',
 'AGR9_E',
 'AGR10_E',
 'CSN1_E',
 'CSN2_E',
 'CSN3_E',
 'CSN4_E',
 'CSN5_E',
 'CSN6_E',
 'CSN7_E',
 'CSN8_E',
 'CSN9_E',
 'CSN10_E',
 'OPN1_E',
 'OPN2_E',
 'OPN3_E',
 'OPN4_E',
 'OPN5_E',
 'OPN6_E',
 'OPN7_E',
 'OPN8_E',
 'OPN9_E',
 '

In [17]:
data_col = data_col[:50]

for c in data_col:
    data = data.withColumn(c, data[c].cast(FloatType()))

In [18]:
data.printSchema()

root
 |-- EXT1: float (nullable = true)
 |-- EXT2: float (nullable = true)
 |-- EXT3: float (nullable = true)
 |-- EXT4: float (nullable = true)
 |-- EXT5: float (nullable = true)
 |-- EXT6: float (nullable = true)
 |-- EXT7: float (nullable = true)
 |-- EXT8: float (nullable = true)
 |-- EXT9: float (nullable = true)
 |-- EXT10: float (nullable = true)
 |-- EST1: float (nullable = true)
 |-- EST2: float (nullable = true)
 |-- EST3: float (nullable = true)
 |-- EST4: float (nullable = true)
 |-- EST5: float (nullable = true)
 |-- EST6: float (nullable = true)
 |-- EST7: float (nullable = true)
 |-- EST8: float (nullable = true)
 |-- EST9: float (nullable = true)
 |-- EST10: float (nullable = true)
 |-- AGR1: float (nullable = true)
 |-- AGR2: float (nullable = true)
 |-- AGR3: float (nullable = true)
 |-- AGR4: float (nullable = true)
 |-- AGR5: float (nullable = true)
 |-- AGR6: float (nullable = true)
 |-- AGR7: float (nullable = true)
 |-- AGR8: float (nullable = true)
 |-- AGR9: fl

In [21]:
data.select('country').show()

+-------+
|country|
+-------+
|     GB|
|     MY|
|     GB|
|     GB|
|     KE|
|     SE|
|     US|
|     MY|
|     GB|
|     FI|
|     UA|
|     PH|
|     FR|
|     GB|
|     AU|
|     IN|
|     CA|
|     NL|
|     ZA|
|     HK|
+-------+
only showing top 20 rows



In [31]:
data_kr = data.filter(data['country']=='KR')
data_us = data.filter(data['country']=='US')

In [34]:
data_country = data_kr.union(data_us)

In [38]:
indexer = StringIndexer(inputCol='country', outputCol='countryidx')
model = indexer.fit(data_country)
data_convert = model.transform(data_country)

In [44]:
data_convert = data_convert.dropna()

In [62]:
one_encoder = OneHotEncoder(inputCol='countryidx', outputCol='countryonehot', dropLast=False)
encoding_df = one_encoder.fit(data_convert).transform(data_convert)


In [46]:
cols = data_convert.columns[:50]

assembler = VectorAssembler(inputCols=cols, outputCol='features')
data_features = assembler.transform(data_convert)
data_features.select('features').show

+--------------------+
|            features|
+--------------------+
|[2.0,1.0,2.0,2.0,...|
|[1.0,5.0,1.0,5.0,...|
|[3.0,1.0,5.0,1.0,...|
|[4.0,1.0,5.0,1.0,...|
|[1.0,5.0,2.0,4.0,...|
+--------------------+
only showing top 5 rows



In [48]:
pca = PCA(k=2, inputCol='features', outputCol='out_features')
model = pca.fit(data_features)
reduced = model.transform(data_features)

22/11/03 18:10:09 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/11/03 18:10:09 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS


22/11/03 18:10:41 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
22/11/03 18:10:41 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


In [49]:
gm = GaussianMixture(featuresCol = 'out_features', k=2, tol=0.0001, seed=10)
model = gm.fit(reduced)
prob = model.transform(reduced)

22/11/03 18:12:45 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
22/11/03 18:12:45 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


In [51]:
prob.select('prediction', 'probability').show(5, truncate=True)

+----------+--------------------+
|prediction|         probability|
+----------+--------------------+
|         1|[0.18056476119614...|
|         0|[0.73798738942873...|
|         1|[0.32214251345458...|
|         1|[0.42634198294001...|
|         1|[0.29429516247064...|
+----------+--------------------+
only showing top 5 rows



In [70]:
prob.columns

['EXT1',
 'EXT2',
 'EXT3',
 'EXT4',
 'EXT5',
 'EXT6',
 'EXT7',
 'EXT8',
 'EXT9',
 'EXT10',
 'EST1',
 'EST2',
 'EST3',
 'EST4',
 'EST5',
 'EST6',
 'EST7',
 'EST8',
 'EST9',
 'EST10',
 'AGR1',
 'AGR2',
 'AGR3',
 'AGR4',
 'AGR5',
 'AGR6',
 'AGR7',
 'AGR8',
 'AGR9',
 'AGR10',
 'CSN1',
 'CSN2',
 'CSN3',
 'CSN4',
 'CSN5',
 'CSN6',
 'CSN7',
 'CSN8',
 'CSN9',
 'CSN10',
 'OPN1',
 'OPN2',
 'OPN3',
 'OPN4',
 'OPN5',
 'OPN6',
 'OPN7',
 'OPN8',
 'OPN9',
 'OPN10',
 'EXT1_E',
 'EXT2_E',
 'EXT3_E',
 'EXT4_E',
 'EXT5_E',
 'EXT6_E',
 'EXT7_E',
 'EXT8_E',
 'EXT9_E',
 'EXT10_E',
 'EST1_E',
 'EST2_E',
 'EST3_E',
 'EST4_E',
 'EST5_E',
 'EST6_E',
 'EST7_E',
 'EST8_E',
 'EST9_E',
 'EST10_E',
 'AGR1_E',
 'AGR2_E',
 'AGR3_E',
 'AGR4_E',
 'AGR5_E',
 'AGR6_E',
 'AGR7_E',
 'AGR8_E',
 'AGR9_E',
 'AGR10_E',
 'CSN1_E',
 'CSN2_E',
 'CSN3_E',
 'CSN4_E',
 'CSN5_E',
 'CSN6_E',
 'CSN7_E',
 'CSN8_E',
 'CSN9_E',
 'CSN10_E',
 'OPN1_E',
 'OPN2_E',
 'OPN3_E',
 'OPN4_E',
 'OPN5_E',
 'OPN6_E',
 'OPN7_E',
 'OPN8_E',
 'OPN9_E',
 '

In [64]:
one_encoder = OneHotEncoder(inputCol='countryidx', outputCol='countryonehot', dropLast=False)
encoding_df = one_encoder.fit(prob).transform(prob)


In [71]:
assembler = VectorAssembler(inputCols=['countryonehot', 'probability'], outputCol='onehot_features')
data_features = assembler.transform(encoding_df)

In [74]:
data_features.columns

['EXT1',
 'EXT2',
 'EXT3',
 'EXT4',
 'EXT5',
 'EXT6',
 'EXT7',
 'EXT8',
 'EXT9',
 'EXT10',
 'EST1',
 'EST2',
 'EST3',
 'EST4',
 'EST5',
 'EST6',
 'EST7',
 'EST8',
 'EST9',
 'EST10',
 'AGR1',
 'AGR2',
 'AGR3',
 'AGR4',
 'AGR5',
 'AGR6',
 'AGR7',
 'AGR8',
 'AGR9',
 'AGR10',
 'CSN1',
 'CSN2',
 'CSN3',
 'CSN4',
 'CSN5',
 'CSN6',
 'CSN7',
 'CSN8',
 'CSN9',
 'CSN10',
 'OPN1',
 'OPN2',
 'OPN3',
 'OPN4',
 'OPN5',
 'OPN6',
 'OPN7',
 'OPN8',
 'OPN9',
 'OPN10',
 'EXT1_E',
 'EXT2_E',
 'EXT3_E',
 'EXT4_E',
 'EXT5_E',
 'EXT6_E',
 'EXT7_E',
 'EXT8_E',
 'EXT9_E',
 'EXT10_E',
 'EST1_E',
 'EST2_E',
 'EST3_E',
 'EST4_E',
 'EST5_E',
 'EST6_E',
 'EST7_E',
 'EST8_E',
 'EST9_E',
 'EST10_E',
 'AGR1_E',
 'AGR2_E',
 'AGR3_E',
 'AGR4_E',
 'AGR5_E',
 'AGR6_E',
 'AGR7_E',
 'AGR8_E',
 'AGR9_E',
 'AGR10_E',
 'CSN1_E',
 'CSN2_E',
 'CSN3_E',
 'CSN4_E',
 'CSN5_E',
 'CSN6_E',
 'CSN7_E',
 'CSN8_E',
 'CSN9_E',
 'CSN10_E',
 'OPN1_E',
 'OPN2_E',
 'OPN3_E',
 'OPN4_E',
 'OPN5_E',
 'OPN6_E',
 'OPN7_E',
 'OPN8_E',
 'OPN9_E',
 '

In [75]:
Correlation.corr(data_features, 'onehot_features', 'pearson').first()

Row(pearson(onehot_features)=DenseMatrix(4, 4, [1.0, -1.0, 0.0067, -0.0067, -1.0, 1.0, -0.0067, 0.0067, 0.0067, -0.0067, 1.0, -1.0, -0.0067, 0.0067, -1.0, 1.0], False))